# 0: Section Overview

In this section, we will impliment a baseline model. This will provide a basic model against which more complex models can be compared. This comparison will help us to gauge the effectiveness of enhancements or modifications made to the model. Additionally, It sets a standard for evaluating whether more sophisticated models actually provide improvements in predictive performance (in our case, this performance metric is measured as the MSE).

# 1: Necessary Imports

In [6]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# 2: Data Access

We will access the data in the same way we accessed the date in section 03-EDA

In [11]:
test_filepath = r"C:\Users\markm\OneDrive\Documents\University\Year 4\dst\GitHub\Assessment_1\Train_and_Test_data\test.xlsx" # Enter your file path for the train data
train_filepath = r"C:\Users\markm\OneDrive\Documents\University\Year 4\dst\GitHub\Assessment_1\Train_and_Test_data\train.xlsx" # Enter your file path for the test data

test_data = pd.read_excel(test_filepath)
train_data = pd.read_excel(train_filepath)

            date iso_code continent      location  Stringency Index  CH Index  \
0     2020-01-05      AND    Europe       Andorra               0.0       0.0   
1     2020-01-06      AND    Europe       Andorra               0.0       0.0   
2     2020-01-07      AND    Europe       Andorra               0.0       0.0   
3     2020-01-08      AND    Europe       Andorra               0.0       0.0   
4     2020-01-09      AND    Europe       Andorra               0.0       0.0   
...          ...      ...       ...           ...               ...       ...   
52257 2024-07-31      ZAF    Africa  South Africa               NaN       NaN   
52258 2024-08-01      ZAF    Africa  South Africa               NaN       NaN   
52259 2024-08-02      ZAF    Africa  South Africa               NaN       NaN   
52260 2024-08-03      ZAF    Africa  South Africa               NaN       NaN   
52261 2024-08-04      ZAF    Africa  South Africa               NaN       NaN   

       Gov Resp Index  Econ

In [ ]:
print(test_data.head())

# 2: Implimenting the baseline model: Linear Regression

Initally, let us train the model on the entire training data. Note that we don't require the first 4 columns (date, iso_code, continent and location). Therefore, the first thing we will do is remove these columns 